In [1]:
import os
os.environ

environ{'_CONDA_SET_PROJ_LIB': '/home/vagrant/anaconda/envs/py3openwind/share/proj',
        'PROJ_LIB': '/home/vagrant/Miniconda3-4.6.14-Linux-x86_64/envs/py3openwind/share/proj',
        'XDG_SESSION_ID': '8',
        'SHELL': '/bin/bash',
        'TERM': 'xterm-color',
        'SSH_CLIENT': '10.0.2.2 58756 22',
        'CONDA_SHLVL': '1',
        'CONDA_PROMPT_MODIFIER': '(py3openwind) ',
        'PYTHONUNBUFFERED': '1',
        'PYTHONBREAKPOINT': 'ipdb.set_trace',
        'USER': 'vagrant',
        'CONDA_EXE': '/home/vagrant/Miniconda3-4.6.14-Linux-x86_64/bin/conda',
        'SSH_AUTH_SOCK': '/tmp/ssh-tFXMxmhDAt/agent.18804',
        '_CE_CONDA': '',
        'CPL_ZIP_ENCODING': 'UTF-8',
        'PATH': '/home/vagrant/Miniconda3-4.6.14-Linux-x86_64/envs/py3openwind/bin:/home/vagrant/Miniconda3-4.6.14-Linux-x86_64/condabin:/home/vagrant/anaconda/envs/py3openwind/bin:/home/vagrant/bin:/home/vagrant/.local/bin:/home/vagrant/anaconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/b

In [2]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'

import django
django.setup()

from django.conf import settings

ModuleNotFoundError: No module named 'geospaas'

In [3]:
from openwind import sar_wind

ImportError: cannot import name 'Nansatmap'

In [4]:
from geo_spaas_wind.utils import wind_from_sar_and_arome_forecast

In [5]:
fn = 'http://nbstds.met.no/thredds/dodsC/NBS/S1B/2019/06/30/IW/S1B_IW_GRDM_1SDV_20190630T051108_20190630T051141_016921_01FD7F_21CF.nc'

In [6]:
from nansat.nansat import Nansat

In [7]:
fn

'http://nbstds.met.no/thredds/dodsC/NBS/S1B/2019/06/30/IW/S1B_IW_GRDM_1SDV_20190630T051108_20190630T051141_016921_01FD7F_21CF.nc'